In [1]:
import numpy as np

In [2]:
x = [1000, 2000, 3000]

In [3]:
numerator = np.exp(x)
denominator = np.sum(numerator)

softmax = numerator / denominator
softmax

/tmp/ipykernel_626257/2174467088.py:1: RuntimeWarning: overflow encountered in exp
  numerator = np.exp(x)
/tmp/ipykernel_626257/2174467088.py:4: RuntimeWarning: invalid value encountered in divide
  softmax = numerator / denominator


array([nan, nan, nan])

## we got overflow! So we need to do safe softmax.

### 1. minus max trick

In [4]:
x_max = np.max(x)
numerator = np.exp(x - x_max)
denominator = np.sum(numerator)
softmax = numerator / denominator
softmax

array([0., 0., 1.])

The reason why this is working because exponent negative is closer to 0 but exponent positive is infinitely increase.

In [5]:
np.exp(1000)

/tmp/ipykernel_626257/2006519253.py:1: RuntimeWarning: overflow encountered in exp
  np.exp(1000)


inf

In [6]:
np.exp(-1000)

0.0

### 2. LSE trick

Log (A/B) = Log (A) − Log (B)

LSE = Log Sum Exponent, which is the denominator, or Log (B).

While Log (A) = Log (exp^A) = A.

but this one you got log softmax.

In [7]:
log_softmax = x - (x_max + np.log(np.sum(np.exp(x - x_max))))
log_softmax

array([-2000., -1000.,     0.])

In [8]:
np.exp(log_softmax)

array([0., 0., 1.])

## Online softmax

If you have huge matrix, calculating softmax is very expensive, actually we can do it by chunk by calculating the global values.

The paper from https://arxiv.org/pdf/1805.02867

In [10]:
-np.inf

-inf

In [13]:
x

[1000, 2000, 3000]

In [16]:
m, d = -np.inf, 0
for x_ in x:
    m_ = max(m, x_)
    d = d * np.exp(m - m_) + np.exp(x_ - m_)
    m = m_

In [19]:
np.exp(np.array(x) - m) / d

array([0., 0., 1.])

### Go bigger

In [36]:
x = np.random.normal(size = (10000))
splitted = np.split(x, 10)
chunks = []
for s in splitted:
    m, d = -np.inf, 0
    for x_ in s:
        m_ = max(m, x_)
        d = d * np.exp(m - m_) + np.exp(x_ - m_)
        m = m_
    chunks.append((m, d))

In [37]:
m_global = -np.inf
for m, _ in chunks:
    m_global = max(m_global, m)

d_global = 0
for m, d in chunks:
    d_global += d * np.exp(m - m_global)

In [39]:
np.sum(np.exp(x - m_global) / d_global)

0.9999999999999996